In [1]:
import open3d as o3d
import numpy as np
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.append('../sample')
from readdatas import *

In [3]:
path = 'D:\project\Point_Datas\Point Cloud Data\Corner.ply'
pcd =  ReadDatas(path)

In [4]:
gpf = GpfGroundExtractor()
gpf.set_data(pcd)
gpf._process_data()

In [5]:
ground = pcd.ground
ground_o3d = gpf.np_to_o3d(pcd.ground)

In [6]:
curvity = gpf.curvature_calculation(ground_o3d)

In [8]:
ground_nor = np.asarray(ground_o3d.normals)

In [8]:
z = [0,0,1]
cos_angle = np.dot(ground_nor, z) / (np.linalg.norm(ground_nor) * np.linalg.norm(z))
angle = np.arccos(cos_angle)

In [9]:
angle = np.degrees(angle)

In [6]:
ground_tree = o3d.geometry.KDTreeFlann(ground_o3d)
n = len(ground)
feature_vector = np.zeros((n,2))
for i in range(n):
    [num, idx, _] = ground_tree.search_radius_vector_3d(ground_o3d.points[i], 0.2)
    point_neighbour = ground[idx,:]
    point_neighbour_max = point_neighbour[:,2].max(axis=0)
    point_neighbour_min = point_neighbour[:,2].min(axis=0)
    point_neighbour_h = point_neighbour_max - point_neighbour_min
    point_neighbour_var = np.var(point_neighbour[:,2],axis=0)
    feature_vector[i,:] = [point_neighbour_h,point_neighbour_var]

In [9]:
feature_vector = np.hstack((feature_vector,ground_nor.reshape(-1,3)))
# feature_vector = np.hstack((feature_vector,curvity.reshape(-1,1)))

In [16]:
colors[:,:] = [1,0,0]
colors[pcd.no_paves] = [0,1,0]

In [17]:
ground_o3d.colors = o3d.utility.Vector3dVector(colors)
o3d.visualization.draw_geometries([ground_o3d])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 不支持请求的转换操作。 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 


In [10]:
dps = DriPathSegment()
dps.set_data(pcd)
dps._process_data()

[Open3D INFO] Picked point #15094 (-14., -13., -1.6) to add in queue.
[Open3D INFO] Picked point #18105 (-4.7, -13., -1.6) to add in queue.
[Open3D INFO] Picked point #47894 (2.3, -10., -1.5) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: 句柄无效。 
[Open3D INFO] Picked point #55616 (5.1, -5.6, -1.5) to add in queue.
[Open3D INFO] Picked point #67993 (5.4, -0.39, -1.5) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #67711 (5.4, 4.3, -1.5) to add in queue.
[Open3D INFO] Picked point #68048 (5.4, 6.0, -1.5) to add in queue.


In [18]:
y = np.full(len(ground),False,dtype=bool)
y[pcd.no_paves] = True

In [19]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [31]:
import joblib
# clf = joblib.load('D:\project\Point_Datas\model.pkl')
joblib.dump(clf, 'SVM_model_0.5.pkl')

['SVM_model_0.5.pkl']

In [28]:
# 创建一个SVM模型，使用RBF核函数
clf = svm.SVC(kernel='rbf', C=1, gamma=0.5)

# 训练模型
clf.fit(feature_vector, y)

SVC(C=1, gamma=0.5)

In [29]:
y_pred = clf.predict(feature_vector[pcd.no_paves])
y_pred_all = clf.predict(feature_vector)

In [30]:
accuracy = accuracy_score(y[pcd.no_paves], y_pred)

# 输出准确率
print("Accuracy: %0.2f" % accuracy)

accuracy_all = accuracy_score(y, y_pred_all)

# 输出准确率
print("Accuracy: %0.2f" % accuracy_all)

Accuracy: 0.96
Accuracy: 1.00
